In [1]:
import numpy as np
import pandas as pd
from os import path,makedirs
from pathlib import Path
import matplotlib.pyplot as plt
import sys
import shelve
import pprint

In [2]:
def ReadFasta(filename):
    with open(filename,'r') as f:
        data = f.readlines()
    return data

def ParseReads(data):
    reads = {}
    for num in range(len(data)):
        if data[num].startswith('>'):
            key = data[num].rstrip() # remove newline char
            val = data[num+1].rstrip() # remove newline char
            reads[key] = val
    return reads

def ConvertToBinary(s):
    out_string = np.array([])
    for letter in s:
        out_string = np.append(out_string,hash(letter))
    return out_string

class GraphNode():
    def __init__(self,seq):
        self.seq = seq
        self.next = []
        self.prev = []
        self.freq = 1

def construct_sequence_from_nodes(node_list):
    out_sequence = ''.join([n.seq[0] for n in node_list[0:-1]])+node_list[-1].seq
    return out_sequence

class DBGraph():
    pass

def RemoveEdge(edge_list,edge_to_remove):
    try:
        edge_list.remove(edge_to_remove)
    except ValueError:
        print('Edge %s not found in list! Please check.')
    return edge_list

def SelectNextNode(current_node,node_table):
    options = np.unique(current_node.next)
    if len(options)>1:
        #node degree of all our possible next nodes
        possible_next_node_degrees = np.array([len(node_table[current_node.seq[1:]+x].next) for x in options])
        #if none of them have any edges, just pick one at random, we will stop here anyways!
        if not np.any(possible_next_node_degrees>0):
            next_na = np.random.choice(options)
        else:
            #try to find ones that aren't bridges--have a degree of more than 1
            try:
                next_na = np.random.choice(options[np.where(possible_next_node_degrees>1)])
            #if they are all bridges, then just pick any that aren't zero
            except ValueError:
                next_na = np.random.choice(options[np.where(possible_next_node_degrees>0)])
    elif len(options) == 1:
        next_na = options[0]
    else:
        next_na = None
    return next_na

In [3]:

reads = ParseReads(ReadFasta('../data/READS.fasta'))

h = np.histogram([len(x) for x in list(reads.values())],bins=100)

In [4]:
h

(array([2066, 1472, 1273, 1120,  988, 1000, 1352, 1075, 1033, 1081, 1116,
        1259, 1752, 1340, 1412, 1534, 1562, 1853, 2634, 2139, 2383, 2523,
        2654, 2866, 2857, 4091, 3140, 3171, 3272, 3368, 3337, 4425, 3200,
        3247, 3037, 3007, 2941, 3790, 2693, 2540, 2427, 2378, 2255, 2855,
        1896, 1701, 1740, 1603, 1506, 1407, 1753, 1147, 1106, 1005,  947,
         861, 1072,  731,  593,  547,  451,  455,  478,  311,  257,  233,
         164,  180,  204,  126,  105,   77,   82,   65,   50,   39,   24,
          21,   17,   15,    9,    4,    9,    2,    2,    4,    0,    1,
           0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           1], dtype=int64),
 array([ 30.  ,  33.16,  36.32,  39.48,  42.64,  45.8 ,  48.96,  52.12,
         55.28,  58.44,  61.6 ,  64.76,  67.92,  71.08,  74.24,  77.4 ,
         80.56,  83.72,  86.88,  90.04,  93.2 ,  96.36,  99.52, 102.68,
        105.84, 109.  , 112.16, 115.32, 118.48, 121.64, 124.8 , 127.96,
        131.12, 1

In [35]:
# reads = ParseReads(ReadFasta('../data/READS.fasta'))
# 
# h = np.histogram([len(x) for x in list(reads.values())],bins=100)
# read_lengths = [len(x) for x in list(reads.values())]
# plt.hist([len(x) for x in list(reads.values())],bins=100)
# 
# %%time
# #read and parse all our k-mers
# kmersize = 27
# node_table = {}
# for read in reads.values():
#     idx = 0;
#     previousNode = None
#     while idx<len(read)-kmersize+1:
#         kmer = read[idx:kmersize+idx]
#         kmer_1 = kmer[0:-1]
#         kmer_2 = kmer[-1]
#         if kmer_1 in node_table:#grab the existing node
#             currentNode = node_table[kmer_1]
#             #add one to the frequency
#             currentNode.freq+=1
#         else:#create new node
#             currentNode = GraphNode(kmer_1)
#         #add our next node
#         currentNode.next.append(kmer_2)
#         #add the previous node if its here
#         if previousNode:
#             currentNode.prev.append(previousNode)
#         #save it to our node table!
#         node_table[kmer_1] = currentNode
#         #set our previous node for reversal of the graph
#         previousNode = kmer_1[0]
#         idx+=1
# 
# outname = '../data/DeBruijneGraph'
# graph = shelve.open(outname)
# graph['node_table'] = node_table
# graph['kmersize'] = kmersize
# graph['reads'] = reads
# graph.close()

In [ ]:
outname = '../data/DeBruijneGraph'
graph = shelve.open(outname)
node_table = graph['node_table']
kmersize = graph['kmersize']
reads = graph['reads']

In [ ]:
# read in our query
query_seq = list(ParseReads(ReadFasta('../data/QUERY.fasta')).values())[0]

In [ ]:
# solve for our query sequence
idx = 0
solved_nodes = []
removed_edges = []
while idx<len(query_seq)-kmersize+2:#+2 because we need the end of the sequece, i.e. we have to dial ONE. NUMBER. HIGHER.
    kmer_1 = query_seq[idx:kmersize+idx-1]
    try:
        cn = node_table[kmer_1]
    except(KeyError):
        raise ValueError('Kmer not found %s'%kmer_1)
    #check that our next node solution is valid!
    if len(query_seq)>(kmersize+idx):
        next_node = query_seq[kmersize+idx-1] 
        if not next_node in cn.next:
            raise ValueError('No solution found! Stopping at this sequence: %s'%construct_sequence_from_nodes(solved_nodes))
        else:
            #remove the edge!
            removed_edges += RemoveEdge(cn.next,next_node)
    solved_nodes.append(cn)    
    idx+=1

In [13]:
#extend our solution!
cn = solved_nodes[-1]
keep_looping = True
while keep_looping:
    next_na = SelectNextNode(cn, node_table)
    #if we can't find a next node, stop looping
    if next_na is None:
        keep_looping = False
    #if we can, look for it in the node table
    else:
        try:
            next_node = node_table[cn.seq[1:]+next_na]
        except KeyError:
            print('Sequence %s not found in the node table!'%(cn.seq[1:]+next_na))
            keep_looping = False
        removed_edges += RemoveEdge(cn.next,next_na)
        solved_nodes.append(cn)
    if len(solved_nodes) == 5000:
        keep_looping = False